In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import t

# Set random seed for reproducibility
np.random.seed(42)

# A primer on predictive inference

## Set-up

$$
(X, Y) \sim P_X \times P_{Y \mid X}
$$

* $X$: *features* or *predictors*
* $Y$: *response*

**Goal.** We want to predict $Y$ given $X$.

**Examples.**
  1. $X =$ father's height, $Y = $ child's height
  2. $X =$ image, $Y = $ label
  3. ...

*Point prediction.* Use available data to train a model $\hat{\mu}$ and do $\hat{\mu}(X) \approx Y$.

***Problem.*** How can we quantify the prediction error in $\hat{\mu}(X)$?

## Prediction interval

A $100 \times (1-\alpha)\%$ *prediction interval* for $Y$ is an interval $\hat{C}$ satisfying

$$
\mathbb{P}\left\{ Y \in \hat{C}(X) \right\} \geq 1-\alpha.
$$

## Classical example: Sir Francis Galton's data on the heights of parents and their children

As a warm-up, let’s consider a classic dataset: Sir Francis Galton’s study on the relationship between the heights of parents and their children. This is a well-studied example that is often used to introduce simple linear regression and the concept of prediction intervals.

A bit of a historical background: In the late 19th century, Sir Francis Galton, a pioneer in statistics and heredity studies, investigated the relationship between parents' and their children's heights. Through his analysis, Galton observed that while tall parents tended to have tall children, the children's heights were, on average, closer to the population mean than their parents' heights. He termed this phenomenon "regression toward mediocrity," which we now know as "regression toward the mean." This insight led to the development of the statistical method we call "regression."

In [ ]:
# Load the data
df = pd.read_csv("Galton.txt", sep="\t")

# Extract the relevant columns
X = df["Father"].astype(float)
Y = df["Height"].astype(float)

# Find the total number of observations and report
n = X.shape[0] 
print("n = {:d}".format(n))

# Split the data 9:1 for training:testing
idx = np.random.permutation(n)
ntr = int(np.floor(0.9*n))
idx_tr, idx_te = idx[:ntr], idx[ntr:]

In [ ]:
# Fit a simple linear regression model using the training set
Xtr_with_intercept = sm.add_constant(X[idx_tr])
model1 = sm.OLS(Y[idx_tr], Xtr_with_intercept).fit()

# Extract prediction and prediction intervals
Xrange = np.linspace(X.min(), X.max(), 100)
Xrange_with_intercept = sm.add_constant(Xrange)
Yhat_summary_frame = model1.get_prediction(Xrange_with_intercept).summary_frame(alpha=0.10)
Yhat = Yhat_summary_frame["mean"]
Yhat_l = Yhat_summary_frame["obs_ci_lower"]
Yhat_u = Yhat_summary_frame["obs_ci_upper"]

# Plot the data and the fitted model & the prediction band
plt.figure(figsize=(8, 6))
plt.scatter(X[idx_tr], Y[idx_tr], alpha=0.5, label="Training Data")
plt.scatter(X[idx_te], Y[idx_te], alpha=0.5, label="Test Data")
plt.plot(Xrange, Yhat, color="red", label="Fitted Model")
plt.fill_between(Xrange, Yhat_l, Yhat_u, color="gray", alpha=0.3, label="90% Prediction Band")
plt.xlabel("Father's Height")
plt.ylabel("Child's Height")
plt.title("Regression of Child's Height on Father's Height")
plt.legend()
plt.show()

# Compute the coverage on the test data and report
Xte_with_intercept = sm.add_constant(X[idx_te])
Ytehat_summary_frame = model1.get_prediction(Xte_with_intercept).summary_frame(alpha=0.10)
Ytehat = Ytehat_summary_frame["mean"]
Ytehat_l = Ytehat_summary_frame["obs_ci_lower"]
Ytehat_u = Ytehat_summary_frame["obs_ci_upper"]
coverage1 = np.mean(np.logical_and(Ytehat_l <= Y[idx_te], Y[idx_te] <= Ytehat_u))
print("Test coverage is {:f}".format(coverage1))

The above prediction band can be computed as

$$
\hat{C}(x) = \hat{\beta}_0+\hat{\beta}_1 x \pm t_{n-2, 1-\alpha/2} \hat{\sigma} \sqrt{1+\frac{1}{n}+\frac{(x-\bar{X})^2}{\sum_{i=1}^{n} (X_i-\bar{X})^2}}
$$

**Question.** What are the assumptions under which this prediction interval is valid?

## Synthetic data example

Now, let's look at a synthetic dataset due to Romano et al. (2019) that obviously violates the assumptions of simple linear regression. Among many things, this dataset is characterized by heteroscedasticity, meaning the variability of the response variable changes across different levels of the predictor. In such cases, the prediction intervals derived from simple linear models become unreliable in the sense that a 90% prediction interval may undercover or overcover.

In [ ]:
def f(x):
    ''' Construct data (1D example)
    '''
    ax = 0*x
    for i in range(len(x)):
        ax[i] = (np.random.poisson(np.sin(x[i])**2+0.1) + 0.03*x[i]*np.random.randn(1))[0]
        ax[i] += (25*(np.random.uniform(0,1,1)<0.01)*np.random.randn(1))[0]
    return ax.astype(np.float32)

# Generate training data
Xtr = np.random.uniform(0, 5.0, size=ntr).astype(np.float32)
Ytr = f(Xtr)

# Generate test data
Xte = np.random.uniform(0, 5.0, size=n-ntr).astype(np.float32)
Yte = f(Xte)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(Xtr, Ytr, alpha=0.5, label="Training Data")
plt.scatter(Xte, Yte, alpha=0.5, label="Test Data")
plt.xlim([0, 5.0])
plt.xlabel("$X$")
plt.ylabel("$Y$")
plt.title("Romano et al. (2019)'s Synthetic Data")
plt.legend()
plt.show()

# Plot the data without outliers (zoom in)
plt.figure(figsize=(8, 6))
plt.scatter(Xtr, Ytr, alpha=0.5, label="Training Data")
plt.scatter(Xte, Yte, alpha=0.5, label="Test Data")
plt.xlim([0, 5.0])
plt.ylim([-2.5, 7])
plt.xlabel("$X$")
plt.ylabel("$Y$")
plt.title("Romano et al. (2019)'s Synthetic Data (Zoomed in)")
plt.legend()
plt.show()

Let's pretend we don't know what we are doing, and fit a simple linear model anyway and compute the prediction band as before.

In [ ]:
# Fit a simple linear regression model using the training set
Xtr_with_intercept = sm.add_constant(Xtr)
model2 = sm.OLS(Ytr, Xtr_with_intercept).fit()

# Extract prediction and prediction intervals
Xrange = np.linspace(0, 5.0, 100)
Xrange_with_intercept = sm.add_constant(Xrange)
Yhat_summary_frame = model2.get_prediction(Xrange_with_intercept).summary_frame(alpha=0.10)
Yhat = Yhat_summary_frame["mean"]
Yhat_l = Yhat_summary_frame["obs_ci_lower"]
Yhat_u = Yhat_summary_frame["obs_ci_upper"]

# Plot the data and the fitted model & the prediction band
plt.figure(figsize=(8, 6))
plt.scatter(Xtr, Ytr, alpha=0.5, label="Training Data")
plt.scatter(Xte, Yte, alpha=0.5, label="Test Data")
plt.plot(Xrange, Yhat, color="red", label="Fitted Model")
plt.fill_between(Xrange, Yhat_l, Yhat_u, color="gray", alpha=0.3, label="90% Prediction Band")
plt.xlim([0, 5.0])
plt.xlabel("$X$")
plt.ylabel("$Y$")
plt.title("OLS Homoscedastic Gaussian Error Prediction Band")
plt.legend()
plt.show()

# Compute the coverage on the test data and report
Xte_with_intercept = sm.add_constant(Xte)
Ytehat_summary_frame = model2.get_prediction(Xte_with_intercept).summary_frame(alpha=0.10)
Ytehat = Ytehat_summary_frame["mean"]
Ytehat_l = Ytehat_summary_frame["obs_ci_lower"]
Ytehat_u = Ytehat_summary_frame["obs_ci_upper"]
coverage2 = np.mean(np.logical_and(Ytehat_l <= Yte, Yte <= Ytehat_u))
print("Test coverage is {:f}".format(coverage2))

*If* the simple linear regression prediction interval had the correct coverage, then it would cover the true test observation about 90% of the time. However, we see that the number is nowhere close.

As mentioned above, the biggest problem with this data is heteroscedasticity, i.e., non-constant variability around the conditional mean. Of course, we can try to do a better job by also trying to model the conditional variance. (In fact, this is precisely what we will do when we revisit this example for the conformalized quantile regression (CQR) later.) However, given additional i.i.d. data, there is an intuitive way to construct a prediction interval that will always have 90% coverage with the same model.

Now, suppose we have a *calibration* dataset which consist of additional $m$ i.i.d. observations generated from the same distribution.

Now, let's do a bit of a thought experiment. Suppose I have $m+1$ i.i.d. random variables
$$
S_1, \dots, S_m, S'
$$

1. What is the probability that $S'$ is the smallest?
2. What is the probability that $S'$ is the second smallest?
3. What is the probability that $S'$ is the $k$-th smallest for any $k$?

Let's change the question.
1. What is the probability that $S'$ is less than or equal to the smallest of $S_1, \dots, S_m$?
2. What is the probability that $S'$ is less than or equal to the second smallest of $S_1, \dots, S_m$?
3. What is the probability that $S'$ is less than or equal to the $k$-th smallest of $S_1, \dots, S_m$?

In general, $$\mathbb{P}\left\{S' \leq S_{(k)}\right\} = \frac{k}{m+1}.$$ This is *always* true without *any* assumptions on the distribution of $S$ as long as the data points are i.i.d. (and more generally, we shall see, exchangeable).

Now, let's return to the synthetic data example. Because the simple linear model $\hat{\mu}$ was fitted using the training data, which are independent of both the test and the calibration data, we have that $$S_1 = |Y_1-\hat{\mu}(X_1)|, \dots, S_m = |Y_m-\hat{\mu}(X_m)|, S' = |Y'-\hat{\mu}(X')|$$ are i.i.d., where $(X_1,Y_1), \dots, (X_m,Y_m)$ are the calibration data and $(X',Y')$ is any point in the test data.

Thus, defining $$\hat{q}_{1-\alpha} = \text{the $\lceil (1-\alpha) (m+1) \rceil$-th smallest of $S_1, \dots, S_m$,}$$ we have $$\mathbb{P}\left\{|Y'-\hat{\mu}(X')| \leq \hat{q}_{1-\alpha}\right\} = \frac{\lceil (1-\alpha) (m+1) \rceil}{m+1} \geq \frac{(1-\alpha)(m+1)}{m+1} = 1-\alpha.$$ However, $$\left\{|Y'-\hat{\mu}(X')| \leq \hat{q}_{1-\alpha}\right\} = \left\{Y' \in \hat\mu(X') \pm \hat{q}_{1-\alpha} \right\}.$$ Therefore, the interval $$\tilde{C}(x) = \hat{\mu}(x) \pm \hat{q}_{1-\alpha}$$ *always* satisfies $$\mathbb{P}\left\{Y' \in \tilde{C}(X')\right\} \geq 1-\alpha.$$ Note that we did not have to make any assumptions on the accuracy of $\hat{\mu}$ or the data distribution.

You will get to look at this argument in more detail when we go over the basic theory of conformal prediction in a couple of weeks. For now, let's just check the idea actually works.

We start by generating calibration data. To keep things simple, I'm reusing $m = n_{\text{tr}}$. However, feel free to experiment with my code and see for yourself that the coverage guarantee has nothing to do with $m$ values.

In [ ]:
m = ntr

# Generate calibration data
Xcal = np.random.uniform(0, 5.0, size=m).astype(np.float32)
Ycal = f(Xcal)

# Compute the nonconformity scores, sort, and get the (1-alpha)(m+1)-th smallest element
S = np.abs(Ycal-model2.predict(sm.add_constant(Xcal)))
S = np.sort(S)
qhat = S[int(np.ceil((1-0.1)*(m+1)))-1] # subtract 1, because Python indices start from 0 not 1

# Plot the new prediction band
plt.figure(figsize=(8, 6))
plt.scatter(Xtr, Ytr, alpha=0.5, label="Training Data")
plt.scatter(Xte, Yte, alpha=0.5, label="Test Data")
plt.plot(Xrange, Yhat, color="red", label="Fitted Model")
plt.fill_between(Xrange, Yhat-qhat, Yhat+qhat, color="gray", alpha=0.3, label="90% Prediction Band")
plt.xlim([0, 5.0])
plt.xlabel("$X$")
plt.ylabel("$Y$")
plt.title("OLS Conformal Prediction Band")
plt.legend()
plt.show()

# Compute the coverage on the test data and report
coverage3 = np.mean(np.abs(Yte-Ytehat) <= qhat)
print("Test coverage is {:f}".format(coverage3))

For these linear regression examples with continuous responses, it was somewhat natural to start by measuring the conformity (or lack thereof) as $$s(x, y) = |y-\hat{\mu}(x)|.$$ This is an example of *nonconformity score* (alternatively, just *conformity score*). For different types of problems, other nonconformity scores will be more natural, e.g., image classification problems.

Roughly speaking, a nonconformity score measures how closely an observation sticks to a given model, in this case $\hat{\mu}$ from the simple linear regression. The more an observation "conforms" to the given model the smaller its nonconformity score, and *vice versa*, and hence, the term *conformal prediction*.

Next time, in the first main tutorial, we will look at an image classification example and how the conformal prediction can be used to provide valid uncertainty quantification for even such unstructured problems.